# Required libraries and class definitions

We use CT scan slices (Slice class) as our base. Each Slice has one or more ROIs (SliceROI) and one Dose Deposition (SliceDose).

In [ ]:
import os, glob
import numpy as np
import scipy.ndimage
import pydicom

import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon, MultiPolygon

class SliceROI:
    """ Stores the information of a given ROI for a Slice 
    
    - mask, polygon and paths use the CT's resolution 
    - mask_ds uses the dose deposition's resolution 
    
    Both are defined as dense 2d numpy arrays for ease of use.
    """
    
    uid = None
    name = None
    color = None
    polygons = None
    paths = None
    mask = None
    mask_ds = None
    
    def __init__(self, uid, name, color):
        self.uid = uid
        self.name = name
        self.color = tuple(float(x)/255 for x in color)
        self.paths = []
        self.polygons = MultiPolygon()
        
    def __str__(self):
        lines = []
        lines.append('  ROI {} {}: {} contours'.format(self.uid, self.name, len(self.paths)))
        return '\n'.join(lines)
    
    def add_contour(self, contour):
        self.polygons = self.polygons.symmetric_difference(Polygon(contour))
        self.paths.append(Path(contour))
                    
    def build_mask(self, shape, radius):
        """ Build a binary mask from the Polygon.
        
        Maybe triangulation or subpixel rasters can give better accuracy.
        
        At first I wanted to use Shapely, but matplotlib.path.contains_point is much faster.
        Shapely is still useful for polygon attributes and contour plotting.
        """

        min_x, min_y, max_x, max_y = self.polygons.bounds
        xp = np.arange(int(np.floor(min_x)), int(np.ceil(max_x)))
        yp = np.arange(int(np.floor(min_y)), int(np.ceil(max_y)))
        xv, yv = np.meshgrid(xp, yp)
        points = np.column_stack((xv.ravel(), yv.ravel()))
        
        poly_contains = []
        for path in self.paths:
            poly_contains.append(path.contains_points(points, radius=radius))
        poly_contains = np.logical_xor.reduce(poly_contains)
        
        self.mask = np.zeros(shape, dtype=np.bool)
        for point, inside in zip(points, poly_contains):
            if inside:
                self.mask[point[1], point[0]] = True
            
                    
class SliceDose:
    """ Stores the information of the Dose Deposition for a Slice 
    
    - original is the dose deposition from the DCM file
    - upsampled is the dose deposition scaled to the CT's resolution
    """
    
    original = None
    upsampled = None
    position = None
    spacing = None
    padding = None
    shift = None
    is_interpolated = False
    
    def __init__(self, original, position, spacing):
        self.original = np.copy(original)
        self.position = position
        self.spacing = spacing

    def upsample(self, ct, ct_position, ct_spacing, order):
        """ Upsample the dose to the CT's resolution.
        Also shift the dose so it's correctly positioned with the CT and ROI contours.
        """
        
        zoom = (self.spacing[0] / ct_spacing[0], self.spacing[1] / ct_spacing[1])
        self.upsampled = scipy.ndimage.zoom(self.original, zoom, order=order)
        
        self.padding = (ct.shape[0] - self.upsampled.shape[0], ct.shape[1] - self.upsampled.shape[1])
        self.upsampled = np.pad(self.upsampled, ((0,self.padding[0]),(0,self.padding[1])), mode='constant', constant_values=0)
        
        self.shift = ((self.position[1] - ct_position[1]) / ct_spacing[1], 
                      (self.position[0] - ct_position[0]) / ct_spacing[0])
        self.upsampled = scipy.ndimage.shift(self.upsampled, self.shift, order=order)
        
                    
class Slice:
    """ Stores all the information for a given CT slice. 
    
    - ct is the CT image from the DCM file
    - ct_ds is the CT image scaled to the dose deposition's resolution 
    """
    
    uid = None
    position = None
    spacing = None
    rois = None
    ct = None
    ct_ds = None
    
    dose = None
    
    def __init__(self, s):
        self.uid = s.SOPInstanceUID
        self.ct = s.pixel_array
        self.position = tuple(float(x) for x in s.ImagePositionPatient)
        self.spacing = tuple(float(x) for x in s.PixelSpacing)
        self.rois = {}
        
    def __str__(self):
        lines = []
        lines.append('UID: {}'.format(self.uid))
        lines.append('Position (x, y, z): {}'.format(self.position))
        lines.append('Number of ROIs: {}'.format(len(self.rois)))
        for uid, roi in self.rois.items():
            lines.append(roi.__str__())
        return '\n'.join(lines)
    
    def add_contour(self, roi_uid, roi_name, roi_color, points):
        """ Inserts a contour into its corresponding ROI """
        if roi_uid not in self.rois:
            self.rois[roi_uid] = SliceROI(roi_uid, roi_name, roi_color)
        transl = []
        for point in points:
            x = (point[0] - self.position[0]) / self.spacing[0]
            y = (point[1] - self.position[1]) / self.spacing[1]
            transl.append(tuple([x, y]))
        self.rois[roi_uid].add_contour(transl)
        
    def build_masks(self, radius=0):
        for uid, roi in self.rois.items():
            roi.build_mask(self.ct.shape, radius)
            
        self.downsample_masks()

    def add_dose(self, dose, position, spacing, order=0):
        self.dose = SliceDose(dose, position, spacing)
        self.dose.upsample(self.ct, self.position, self.spacing, order)
        self.downsample_masks()
            
    def downsample_masks(self, order=0):
        """ Downsamples the CT and ROI masks to the dose deposition's resolution.
        Also shifts them so they are correctly positioned.
        """
        
        if self.dose is not None:
            zoom = (self.spacing[0] / self.dose.spacing[0], self.spacing[1] / self.dose.spacing[1])
            shift = ((self.dose.position[1] - self.position[1]) / self.spacing[1], 
                     (self.dose.position[0] - self.position[0]) / self.spacing[0])

            self.ct_ds = scipy.ndimage.shift(self.ct, (-shift[0], -shift[1]), order=order)
            self.ct_ds = self.ct_ds[:-self.dose.padding[0], :-self.dose.padding[1]]
            self.ct_ds = scipy.ndimage.zoom(self.ct_ds, zoom, order=order)
            
            for uid, roi in self.rois.items():
                if roi.mask is not None:
                    roi.mask_ds = scipy.ndimage.shift(roi.mask, (-shift[0], -shift[1]), order=order)
                    roi.mask_ds = roi.mask_ds[:-self.dose.padding[0], :-self.dose.padding[1]]
                    roi.mask_ds = scipy.ndimage.zoom(roi.mask_ds, zoom, order=order)
                    #roi.mask_ds = cv2.resize(roi.mask_ds, dsize=(self.dose.original.shape[1],self.dose.original.shape[0]), interpolation=cv2.INTER_AREA)

                    
def stats(roi_doses):
    """ Just a small function to print statistics :)"""
    lines = [];
    header = "{:^16} {:^9} {:^9} {:^9} {:^9}".format('Region', 'Size', 'Min', 'Avg', 'Max')
    lines.append('-'*len(header))
    lines.append(header)
    lines.append('-'*len(header))
    for name, doses in roi_doses.items():
        lines.append("{:<16} {:>9} {:>9.4f} {:>9.4f} {:>9.4f}".format(name, len(doses), np.min(doses), np.average(doses), np.max(doses)))
    return lines

# Read files and create objects

Please modify these variables:

- ```ct_folder```: Folder where you have the CT files.
- ```rs_filename```: RS file path.
- ```rd_filename```: RD file path.

In [ ]:
ct_folder = '../DICOM/16/ct'
rs_filename = '../DICOM/16/RS.1.2.246.352.205.5438954218386416801.8699273283688404648.dcm'
rd_filename = '../DICOM/16/plan_start/RD.1.2.246.352.71.7.59109062985.6240060.20170830160733.dcm'

slices = {}

filenames = glob.glob(os.path.join(ct_folder, '*'))
print('Reading CT files...', end=' ')
for filename in filenames:
    s = pydicom.dcmread(filename)
    slices[s.SOPInstanceUID] = Slice(s)
print("Loaded {} slices.".format(len(slices.values())))

print('Reading RS file...')
rs = pydicom.dcmread(rs_filename)
roi_names = {}
for roi in rs.StructureSetROISequence:
    roi_names[int(roi.ROINumber)] = roi.ROIName
    
for rcs in rs.ROIContourSequence:
    roi_uid = int(rcs.ReferencedROINumber)
    roi_color = rcs.ROIDisplayColor
    for cs in rcs.ContourSequence:
        points = []
        for i in range(0, len(cs.ContourData), 3):
            points.append(tuple(float(x) for x in cs.ContourData[i:i+3]))
        for cis in cs.ContourImageSequence:
            slice_uid = cis.ReferencedSOPInstanceUID
            slices[slice_uid].add_contour(roi_uid, roi_names[roi_uid], roi_color, points)

print('Reading RD file...', end=' ')
rd = pydicom.dcmread(rd_filename)
doses = rd.pixel_array * float(rd.DoseGridScaling)
dose_v_max = np.max(doses)

position = rd.ImagePositionPatient
spacing = rd.PixelSpacing
frame_offset = np.array(rd.GridFrameOffsetVector) + position[2]

sorted_slices = list(slices.values())
sorted_slices.sort(key=lambda x: x.position[2])
for s in sorted_slices:
    for i in np.where(frame_offset == s.position[2])[0]:
        s.add_dose(doses[i,:,:], position, spacing)
print('Found dose depositions for {} of {} slices.'.format(sum(s.dose is not None for s in sorted_slices), len(sorted_slices)))
        
print('Interpolating missing depositions...')
for i in range(len(sorted_slices)):
    if sorted_slices[i].dose is None:
        dose_a = sorted_slices[i-1].dose
        dose_b = sorted_slices[i+1].dose
        if dose_a is not None and dose_b is not None:
            avg_dose = (dose_a.original + dose_b.original)/2
            sorted_slices[i].add_dose(avg_dose, dose_a.position, dose_a.spacing)
            sorted_slices[i].dose.is_interpolated = True
        else:
            print('Slice {} has no neighbors with valid dose depositions.'.format(position))

for i in range(len(sorted_slices)):
    print("Building ROI masks for slice {}/{}...".format(i+1, len(sorted_slices)), end='\r')
    sorted_slices[i].build_masks()
print("\nAll masks built!")

# Calculate ROI doses using upsampled depositions

In [ ]:
roi_doses = {}

for uid, name in roi_names.items():
    doses = []
    for s in sorted_slices:
        if s.dose is not None and uid in s.rois:
            dose = s.dose.upsampled * s.rois[uid].mask
            doses.extend(dose[dose > 0])
    roi_doses[name] = doses

for line in stats(roi_doses):
    print(line)

# Calculate ROI doses using downsampled masks

In [ ]:
roi_doses = {}

for uid, name in roi_names.items():
    doses = []
    for s in sorted_slices:
        if s.dose is not None and uid in s.rois and not s.dose.is_interpolated:
            # We only use original depositions, ignore interpolated ones.
            dose = s.dose.original * s.rois[uid].mask_ds
            doses.extend(dose[dose > 0])
    roi_doses[name] = doses

for line in stats(roi_doses):
    print(line)